<a href="https://colab.research.google.com/github/MCygert/preparing_for_tensorflow_certificate/blob/main/08_intro_to_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP fundamentals

In [1]:
!nvidia-smi

Sat Jan 22 15:13:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-01-22 15:13:57--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-01-22 15:13:57 (102 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2022-01-22 15:14:00--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.97.128, 142.251.107.128, 74.125.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-01-22 15:14:00 (155 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Let's visualize random data
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")


Target: 0 (not real disaster)
Text:
According to prophecy and also CNN a Mac tablet will completely obliterate the need for other gadgets. CombiningÛ_ http://t.co/xfccvMXuWb

---

Target: 0 (not real disaster)
Text:
Has body bagged ** RT @d_lac: Drake is body bagging meek

---

Target: 0 (not real disaster)
Text:
@ConnorFranta #AskConnor if you were a natural disaster what would you be?

---

Target: 0 (not real disaster)
Text:
@soapscoop i need you to confirm that ross is dead cause i dont trust anyone else yh

---

Target: 0 (not real disaster)
Text:
@libraryeliza he did get a @taylorswift13 'bump' of approval which is probably why he's blown up http://t.co/KOLMzBz1pZ #MusicAdvisory

---



## Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# Use train_test_split to split training data into trainnig and validaiton sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [12]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [14]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000
max_length = 15 
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)


In [17]:
text_vectorizer.adapt(train_sentences)

In [18]:
sample_sentence = " There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_words = words_in_vocab[:5]
bottom_words = words_in_vocab[-5:]
top_words, bottom_words

(['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

### Creating an Embedding using Tensorflow layer

In [20]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 
embedding

In [21]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \n\nEmbeded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 The greatest female beat boxer ever now but it's w/e... Save babies outta burning buildings on my free time but ya know.. whatevs.. 

Embeded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00874851,  0.01935159,  0.01637684, ..., -0.02494843,
          0.04980513, -0.02181329],
        [ 0.01882465, -0.00622703, -0.02724373, ...,  0.04323551,
          0.00921209, -0.04654564],
        [ 0.01784864,  0.02954289,  0.01001159, ..., -0.01980584,
          0.04223252,  0.02273322],
        ...,
        [-0.01294522,  0.03771092,  0.04369161, ...,  0.04395783,
         -0.03724374, -0.00288577],
        [ 0.03370101, -0.00421255, -0.03183972, ...,  0.0332592 ,
          0.02004769,  0.01290968],
        [ 0.04201302, -0.01870262,  0.04026885, ..., -0.02951393,
         -0.01829286, -0.0047605 ]]], dtype=float32)>

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

# Fit the pipeline
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [23]:
# Evaluate our baseline model
baseline_scores = model_0.score(val_sentences, val_labels)
baseline_scores

0.7926509186351706

In [24]:
baseline_preds  = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [25]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {'accuracy': model_accuracy,
                   'precision': model_precision,
                   'recall': model_recall,
                   'f1': model_f1}
  return model_results


In [26]:
# Get baseline results
baseline_results = calculate_results(val_labels, baseline_preds)

In [27]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

## Feed forward model

In [28]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard lgos
SAVE_DIR = 'model_logs'


In [29]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [30]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [31]:
# Compile model
model_1.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [32]:
#Fit the model
model_1_history = model_1.fit(train_sentences,
                                  train_labels,
                                  epochs=5,
                                  validation_data=(val_sentences, val_labels),
                                  callbacks=[create_tensorboard_callback(dir_name='model_logs', experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220122-151404
Epoch 1/5
215/215 [==============================] - 3s 5ms/step - loss: 0.6105 - accuracy: 0.6935 - val_loss: 0.5354 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4409 - accuracy: 0.8192 - val_loss: 0.4699 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3463 - accuracy: 0.8578 - val_loss: 0.4646 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2836 - accuracy: 0.8899 - val_loss: 0.4623 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2373 - accuracy: 0.9143 - val_loss: 0.4837 - val_accuracy: 0.7887


In [33]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4837 - accuracy: 0.7887


[0.48368221521377563, 0.7887139320373535]

In [34]:
model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [35]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [36]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.87139107611549,
 'f1': 0.7852145092095362,
 'precision': 0.7953441303708004,
 'recall': 0.7887139107611548}

In [37]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [38]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [39]:
# Get the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:15]

(10000,
 ['',
  '[UNK]',
  'the',
  'a',
  'in',
  'to',
  'of',
  'and',
  'i',
  'is',
  'for',
  'on',
  'you',
  'my',
  'with'])

In [40]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [41]:
embed_weights = model_1.get_layer('embedding_1').get_weights()[0]
embed_weights, embed_weights.shape

(array([[ 0.01743176,  0.06139462, -0.01014978, ...,  0.06870439,
         -0.01774161,  0.01041661],
        [-0.00066163,  0.05227435,  0.03091203, ...,  0.05831363,
         -0.02647651, -0.01574972],
        [ 0.02273466,  0.03270198,  0.00215714, ..., -0.01130478,
          0.06191296, -0.03456916],
        ...,
        [ 0.04008589,  0.02306705,  0.03053773, ...,  0.00929376,
          0.03373634,  0.03925172],
        [-0.00334276,  0.0317192 , -0.01827867, ...,  0.03856064,
          0.05864128, -0.03091507],
        [ 0.01465977,  0.04256891, -0.03281112, ...,  0.01231664,
          0.01889439, -0.10033177]], dtype=float32), (10000, 128))

In [43]:
 import io

# Create output writers
out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
  if num == 0: 
     continue # skip padding token
  vec = embed_weights[num]
  out_m.write(word + "\n") # write words to file
  out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
out_v.close()
out_m.close()
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download("embedding_vectors.tsv")
  files.download("embedding_metadata.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Network (RNN)

In [63]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x)
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs,outputs, name='model_2_LSTM')

In [64]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_6 (LSTM)               (None, 64)                49408     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [65]:
model_2.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])


In [66]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220122-155816
Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.1259 - accuracy: 0.9564 - val_loss: 0.9942 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0562 - accuracy: 0.9762 - val_loss: 1.0432 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0474 - accuracy: 0.9777 - val_loss: 1.2570 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0438 - accuracy: 0.9796 - val_loss: 1.3725 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0405 - accuracy: 0.9807 - val_loss: 1.4368 - val_accuracy: 0.7585


In [67]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[9.0010482e-01],
       [7.4753463e-01],
       [9.9992108e-01],
       [1.9445066e-01],
       [1.2979830e-04],
       [9.9987614e-01],
       [9.1620249e-01],
       [9.9994266e-01],
       [9.9992383e-01],
       [9.9973649e-01]], dtype=float32)

In [68]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [69]:
model_2_results = calculate_results(val_labels,
                                    model_2_preds)
model_2_results

{'accuracy': 75.8530183727034,
 'f1': 0.7586340057007437,
 'precision': 0.7587797021492048,
 'recall': 0.7585301837270341}

In [70]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [78]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x = layers.LSTM(42, return_sequences=True)(x)
# x = layers.GRU(99)(x)
# x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')


In [79]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru_7 (GRU)                 (None, 64)                37248     
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [80]:
model_3.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [81]:
model_3_history = model_3.fit(train_sentences, train_labels, epochs=5, validation_data=(val_sentences, val_labels), callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220122-161945
Epoch 1/5
215/215 [==============================] - 3s 7ms/step - loss: 0.1365 - accuracy: 0.9467 - val_loss: 0.9507 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0603 - accuracy: 0.9772 - val_loss: 1.0103 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0524 - accuracy: 0.9793 - val_loss: 1.0210 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0442 - accuracy: 0.9800 - val_loss: 1.2156 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0374 - accuracy: 0.9820 - val_loss: 1.4990 - val_accuracy: 0.7598


In [84]:
model_3_preds_probs = model_3.predict(val_sentences)
model_3_preds_probs[:10]

array([[3.8264543e-04],
       [8.5936463e-01],
       [9.9978930e-01],
       [3.2945541e-01],
       [6.3116306e-05],
       [9.9959546e-01],
       [9.8925948e-01],
       [9.9991703e-01],
       [9.9986172e-01],
       [9.9787569e-01]], dtype=float32)

In [85]:
model_3_preds = tf.squeeze(tf.round(model_3_preds_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [87]:
model_3_result = calculate_results(val_labels, model_3_preds)
model_3_result

{'accuracy': 75.98425196850394,
 'f1': 0.7590768821741806,
 'precision': 0.7594780880165999,
 'recall': 0.7598425196850394}

In [88]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

## Model using bidirectional-LSTM RNN 

In [93]:
from tensorflow.keras import layers 
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name='model_4_bidirectional')


In [95]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 15, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                             

In [96]:
# Compile
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [97]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])

Saving TensorBoard log files to: model_logs/bidirectional_RNN/20220122-174503
Epoch 1/5
215/215 [==============================] - 9s 16ms/step - loss: 0.0863 - accuracy: 0.9714 - val_loss: 1.3008 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0411 - accuracy: 0.9807 - val_loss: 1.3242 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0344 - accuracy: 0.9812 - val_loss: 1.6113 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0397 - accuracy: 0.9804 - val_loss: 1.2853 - val_accuracy: 0.7533
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0353 - accuracy: 0.9813 - val_loss: 1.5106 - val_accuracy: 0.7651


In [100]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[7.1170777e-03],
       [7.5573194e-01],
       [9.9997175e-01],
       [2.8381044e-01],
       [7.2697563e-05],
       [9.9980134e-01],
       [9.9310666e-01],
       [9.9998057e-01],
       [9.9996710e-01],
       [9.9775666e-01]], dtype=float32)

In [101]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [102]:
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 76.50918635170603,
 'f1': 0.7644302589305015,
 'precision': 0.76472972449374,
 'recall': 0.7650918635170604}